# Create embeddings using Amazon Titan Text Embeddings model and run similarity tests

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Setup

In [1]:
import json
import os
import sys

import boto3
import botocore

import numpy as np
from numpy.linalg import norm

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock

boto3_bedrock = boto3.client(service_name='bedrock-runtime')

modelId="amazon.titan-embed-text-v1"

## Define a resuable Python function to generate the embeddings for each sentence

In [2]:
def sen2vec(sentence):
    # Convert the given sentence to a vector representation using a text embedding model.  
    input_body = {"inputText": sentence}
    try:
        response = boto3_bedrock.invoke_model(
            body=json.dumps(input_body),
            modelId=modelId,
            accept="application/json",
            contentType="application/json",
            )
        response_body = json.loads(response.get("body").read())
        vector = np.array(response_body.get("embedding"))            
        return vector
    except Exception as e:
        print(e)

## Generate and store the embeddings in an array

In [3]:
with open("documents.txt") as doc:
    num_records = len(doc.readlines())
    
doc_array=np.empty(shape=(num_records), dtype="S255")
embed_array = np.zeros(shape=(num_records, 1536))
    
with open("documents.txt") as doc:
    for num, line in enumerate(doc, 0):
      doc = line.strip('\n')
      doc_array[num] = doc
      embed_array[num] = sen2vec(doc)

print(embed_array.shape)
print(embed_array)

(6, 1536)
[[-1.80664062e-01  3.76953125e-01  7.37304688e-02 ...  8.98437500e-02
  -7.92968750e-01  2.45117188e-01]
 [-3.08593750e-01  5.11718750e-01  1.58203125e-01 ...  1.15234375e-01
  -9.53125000e-01  3.59375000e-01]
 [ 1.68945312e-01  4.53125000e-01  2.25585938e-01 ... -5.44433594e-02
  -8.12500000e-01 -3.41796875e-02]
 [ 2.44140625e-01  3.61328125e-01  1.12304688e-01 ... -6.54296875e-02
  -8.86718750e-01  3.57421875e-01]
 [ 1.37695312e-01  6.98089600e-04  6.78710938e-02 ... -3.07617188e-02
  -5.15625000e-01  2.92968750e-01]
 [-2.75878906e-02  6.40625000e-01  2.30468750e-01 ... -6.20117188e-02
  -9.29687500e-01  2.22656250e-01]]


## Interactively query using similarity search

Create an interactive query response loop that uses similarity search to pull relevent documents from the matrix vector store. We use 0.5 as the similarity threshold to limit hallucination. Queries that relate to the course names in the document store will lead to successful results. If none of the results exceed the 0.5 threshold, "I don't know" will be the response.

In [ ]:
while True:
    query = input("\nEnter your query or say quit to exit: ")
    if query == "quit":
        break
    else:
        embed_query = sen2vec(query)
        denominator = norm(embed_array, axis=1) * norm(embed_query)
        similarity = embed_array.dot(embed_query) / denominator
        max_value = max(similarity)
        max_value_index = similarity.argmax()
        print("\nQuery: " + query)
        if max_value > 0.5:
            print("Document that can answer the question: " + doc_array[max_value_index].decode('UTF-8'))
        else:
            print("Answer: " + "I don't know")
        print("\nSimilarity vector used for document selection")
        print(similarity)


Enter your query or say quit to exit:  What is the price of a Google Nexus 10



Query: What is the price of a Google Nexus 10
Answer: I don't know

Similarity vector used for document selection
[0.46151531 0.36395106 0.38836125 0.41931347 0.32943636 0.39567247]



Enter your query or say quit to exit:  What is the price of product number 1982



Query: What is the price of product number 1982
Document that can answer the question: The price of product number 1982 is 63$

Similarity vector used for document selection
[0.68779493 0.68348647 0.6969193  0.7541765  0.90827418 0.71920015]



Enter your query or say quit to exit:  what is the price of product number 9823



Query: what is the price of product number 9823
Document that can answer the question: The price of product number 9823 is 33$

Similarity vector used for document selection
[0.6958919  0.70851048 0.70752366 0.87945729 0.66044613 0.74153954]



Enter your query or say quit to exit:  what is the price of Iphone 16



Query: what is the price of Iphone 16
Answer: I don't know

Similarity vector used for document selection
[0.40624197 0.38725182 0.38570658 0.42092556 0.360412   0.41923814]
